## 분석목적: nusc 데이터 전처리 코드 분석

In [1]:
import argparse
import json
import os
import sys

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

import numpy as np
import torch
import yaml
from det3d.datasets import build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
import torch.distributed as dist
import subprocess

# 주피터 노트북 경로설정
os.chdir('../')


def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", default="configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep.py", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument("--resume_from", help="the checkpoint file to resume from")
    parser.add_argument(
        "--validate",
        action="store_true",
        help="whether to evaluate the checkpoint during training",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument("--seed", type=int, default=None, help="random seed")
    parser.add_argument(
        "--launcher",
        choices=["pytorch", "slurm"],
        default="pytorch",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument(
        "--autoscale-lr",
        action="store_true",
        help="automatically scale lr with the number of gpus",
    )
    args = parser.parse_args(args=[])
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args
args = parse_args()

no apex
No Tensorflow
Deformable Convolution not built!
No APEX!


In [2]:
cfg = Config.fromfile(args.config)
cfg.local_rank = args.local_rank
distributed = False
logger = get_root_logger(cfg.log_level)
logger.info("Distributed training: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")

2024-04-03 15:30:13,683 - INFO - Distributed training: False
2024-04-03 15:30:13,683 - INFO - torch.backends.cudnn.benchmark: False


In [3]:
cfg.data.train

{'type': 'NuScenesDataset',
 'root_path': 'data/nuScenes',
 'info_path': 'data/nuScenes/infos_train_10sweeps_withvelo_filter_True.pkl',
 'ann_file': 'data/nuScenes/infos_train_10sweeps_withvelo_filter_True.pkl',
 'nsweeps': 10,
 'class_names': ['car',
  'truck',
  'construction_vehicle',
  'bus',
  'trailer',
  'barrier',
  'motorcycle',
  'bicycle',
  'pedestrian',
  'traffic_cone'],
 'pipeline': [{'type': 'LoadPointCloudFromFile', 'dataset': 'NuScenesDataset'},
  {'type': 'LoadPointCloudAnnotations', 'with_bbox': True},
  {'type': 'Preprocess',
   'cfg': {'mode': 'train',
    'shuffle_points': True,
    'global_rot_noise': [-0.3925, 0.3925],
    'global_scale_noise': [0.95, 1.05],
    'db_sampler': {'type': 'GT-AUG',
     'enable': False,
     'db_info_path': 'data/nuScenes/dbinfos_train_10sweeps_withvelo.pkl',
     'sample_groups': [{'car': 2},
      {'truck': 3},
      {'construction_vehicle': 7},
      {'bus': 4},
      {'trailer': 6},
      {'barrier': 2},
      {'motorcycle': 6}

## build_dataset

In [4]:
from det3d.datasets.registry import DATASETS

args = cfg.data.train.copy()
obj_type = args.pop("type")


In [5]:
for k, v in args.items():
    print(f"{k}")

root_path
info_path
ann_file
nsweeps
class_names
pipeline


In [6]:
obj_cls = DATASETS.get(obj_type)
default_args = dict(train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
if default_args is not None:
    for name, value in default_args.items():
        args.setdefault(name, value)
datasets = [obj_cls(**args)]

2024-04-03 15:31:08,551 - INFO - {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}
2024-04-03 15:31:08,551 - INFO - [-1]
2024-04-03 15:31:10,389 - INFO - load 62964 traffic_cone database infos
2024-04-03 15:31:10,390 - INFO - load 65262 truck database infos
2024-04-03 15:31:10,390 - INFO - load 339949 car database infos
2024-04-03 15:31:10,390 - INFO - load 161928 pedestrian database infos
2024-04-03 15:31:10,391 - INFO - load 26297 ignore database infos
2024-04-03 15:31:10,391 - INFO - load 11050 construction_vehicle database infos
2024-04-03 15:31:10,391 - INFO - load 107507 barrier database infos
2024-04-03 15:31:10,391 - INFO - load 8846 motorcycle database infos
2024-04-03 15:31:10,392 - INFO - load 8185 bicycle database infos
2024-04-03 15:31:10,392 - INFO - load 12286 bus database infos
2024-04-03 15:31:10,392 - INFO - load 19202 trailer database infos
2024-04-03 15:31:11,208

10


## NuScenesDataset

In [7]:
# obj_cls = DATASETS.get(obj_type) => det3d.datasets.nuscenes.nuscenes.NuScenesDataset

In [7]:
# __init__
import pickle
general_to_detection = {
    "human.pedestrian.adult": "pedestrian",
    "human.pedestrian.child": "pedestrian",
    "human.pedestrian.wheelchair": "ignore",
    "human.pedestrian.stroller": "ignore",
    "human.pedestrian.personal_mobility": "ignore",
    "human.pedestrian.police_officer": "pedestrian",
    "human.pedestrian.construction_worker": "pedestrian",
    "animal": "ignore",
    "vehicle.car": "car",
    "vehicle.motorcycle": "motorcycle",
    "vehicle.bicycle": "bicycle",
    "vehicle.bus.bendy": "bus",
    "vehicle.bus.rigid": "bus",
    "vehicle.truck": "truck",
    "vehicle.construction": "construction_vehicle",
    "vehicle.emergency.ambulance": "ignore",
    "vehicle.emergency.police": "ignore",
    "vehicle.trailer": "trailer",
    "movable_object.barrier": "barrier",
    "movable_object.trafficcone": "traffic_cone",
    "movable_object.pushable_pullable": "ignore",
    "movable_object.debris": "ignore",
    "static_object.bicycle_rack": "ignore",
}
def load_infos(info_path, class_names, load_interval, test_mode):

    with open(info_path, "rb") as f:
        _nusc_infos_all = pickle.load(f)

    _nusc_infos_all = _nusc_infos_all[::load_interval]

    if not test_mode:  # if training
        frac = int(len(_nusc_infos_all) * 0.25)

        _cls_infos = {name: [] for name in class_names}
        for info in _nusc_infos_all:
            for name in set(info["gt_names"]):
                if name in class_names:
                    _cls_infos[name].append(info)

        duplicated_samples = sum([len(v) for _, v in _cls_infos.items()])
        _cls_dist = {k: len(v) / max(duplicated_samples, 1) for k, v in _cls_infos.items()}

        _nusc_infos = []

        frac = 1.0 / len(class_names)
        ratios = [frac / v for v in _cls_dist.values()]

        for cls_infos, ratio in zip(list(_cls_infos.values()), ratios):
            _nusc_infos += np.random.choice(
                cls_infos, int(len(cls_infos) * ratio)
            ).tolist()

        _cls_infos = {name: [] for name in class_names}
        for info in _nusc_infos:
            for name in set(info["gt_names"]):
                if name in class_names:
                    _cls_infos[name].append(info)

        _cls_dist = {
            k: len(v) / len(_nusc_infos) for k, v in _cls_infos.items()
        }
    else:
        if isinstance(_nusc_infos_all, dict):
            _nusc_infos = []
            for v in _nusc_infos_all.values():
                _nusc_infos.extend(v)
        else:
            _nusc_infos = _nusc_infos_all
    
    return _nusc_infos

pipeline = args["pipeline"]
class_names = args["class_names"]
test_mode = False
version="v1.0-trainval"
load_interval = 1
nsweeps = args["nsweeps"]
root_path = args["root_path"]
assert nsweeps > 0
_info_path = args["info_path"]
_class_names = args["class_names"]
_num_point_features = 5
virtual = args.get('virtual', False)
eval_version = "detection_cvpr_2019"

_nusc_infos = load_infos(_info_path, _class_names, load_interval, test_mode)

logger.info(f"pipeline: {pipeline}")
logger.info(f"class_names: {class_names}")
logger.info(f"nsweeps: {nsweeps}")
logger.info(f"root_path: {root_path}")
logger.info(f"_info_path: {_info_path}")
logger.info(f"_class_names: {_class_names}")

2024-04-03 15:31:18,948 - INFO - pipeline: [{'type': 'LoadPointCloudFromFile', 'dataset': 'NuScenesDataset'}, {'type': 'LoadPointCloudAnnotations', 'with_bbox': True}, {'type': 'Preprocess', 'cfg': {'mode': 'train', 'shuffle_points': True, 'global_rot_noise': [-0.3925, 0.3925], 'global_scale_noise': [0.95, 1.05], 'db_sampler': {'type': 'GT-AUG', 'enable': False, 'db_info_path': 'data/nuScenes/dbinfos_train_10sweeps_withvelo.pkl', 'sample_groups': [{'car': 2}, {'truck': 3}, {'construction_vehicle': 7}, {'bus': 4}, {'trailer': 6}, {'barrier': 2}, {'motorcycle': 6}, {'bicycle': 6}, {'pedestrian': 2}, {'traffic_cone': 2}], 'db_prep_steps': [{'filter_by_min_num_points': {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}}, {'filter_by_difficulty': [-1]}], 'global_random_rotation_range_per_object': [0, 0], 'rate': 1.0}, 'class_names': ['car', 'truck', 'construction_vehicle', 'bus', 'trailer

In [8]:
logger.info(f"len(_nusc_infos): {len(_nusc_infos)}")
for k, v in _nusc_infos[0].items():
    logger.info(f"key of _nusc_infos: {k}")
_nusc_infos[0]

2024-04-03 15:31:18,955 - INFO - len(_nusc_infos): 123580
2024-04-03 15:31:18,955 - INFO - key of _nusc_infos: lidar_path
2024-04-03 15:31:18,956 - INFO - key of _nusc_infos: cam_front_path
2024-04-03 15:31:18,956 - INFO - key of _nusc_infos: cam_intrinsic
2024-04-03 15:31:18,956 - INFO - key of _nusc_infos: token
2024-04-03 15:31:18,956 - INFO - key of _nusc_infos: sweeps
2024-04-03 15:31:18,957 - INFO - key of _nusc_infos: ref_from_car
2024-04-03 15:31:18,957 - INFO - key of _nusc_infos: car_from_global
2024-04-03 15:31:18,957 - INFO - key of _nusc_infos: timestamp
2024-04-03 15:31:18,957 - INFO - key of _nusc_infos: all_cams_from_lidar
2024-04-03 15:31:18,957 - INFO - key of _nusc_infos: all_cams_intrinsic
2024-04-03 15:31:18,957 - INFO - key of _nusc_infos: all_cams_path
2024-04-03 15:31:18,958 - INFO - key of _nusc_infos: gt_boxes
2024-04-03 15:31:18,958 - INFO - key of _nusc_infos: gt_boxes_velocity
2024-04-03 15:31:18,958 - INFO - key of _nusc_infos: gt_names
2024-04-03 15:31:18

{'lidar_path': '/mnt/hdd0/nuScenes/samples/LIDAR_TOP/n015-2018-08-03-15-00-36+0800__LIDAR_TOP__1533279699399021.pcd.bin',
 'cam_front_path': '/mnt/hdd0/nuScenes/samples/CAM_FRONT/n015-2018-08-03-15-00-36+0800__CAM_FRONT__1533279699362460.jpg',
 'cam_intrinsic': array([[1.26641720e+03, 0.00000000e+00, 8.16267020e+02],
        [0.00000000e+00, 1.26641720e+03, 4.91507066e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'token': 'c3d61b8b66424ed9b1faa5114a0749e8',
 'sweeps': [{'lidar_path': '/mnt/hdd0/nuScenes/sweeps/LIDAR_TOP/n015-2018-08-03-15-00-36+0800__LIDAR_TOP__1533279699349247.pcd.bin',
   'sample_data_token': 'fb4556116e844a559d9310d1073e9a1e',
   'transform_matrix': array([[ 9.99746766e-01,  2.24956666e-02,  5.89971196e-04,
            1.35732038e-02],
          [-2.24952384e-02,  9.99746689e-01, -7.22662026e-04,
           -1.73858174e-01],
          [-6.06078514e-04,  7.09207481e-04,  9.99999565e-01,
           -3.80722449e-03],
          [ 0.00000000e+00,  0.

### compose 초기화

In [9]:
# Compose 
import collections
from det3d.utils import build_from_cfg
from det3d.datasets.registry import PIPELINES
class Compose(object):
    def __init__(self, transforms):
        assert isinstance(transforms, collections.abc.Sequence)
        self.transforms = []
        for transform in transforms:
            if isinstance(transform, dict):
                if transform['type'] == 'Empty':
                    continue 
                transform = build_from_cfg(transform, PIPELINES)
                self.transforms.append(transform)
            elif callable(transform):
                self.transforms.append(transform)
            else:
                raise TypeError("transform must be callable or a dict")

    def __call__(self, res, info):
        for t in self.transforms:
            res, info = t(res, info)
            if res is None:
                return None
        return res, info

    def __repr__(self):
        format_string = self.__class__.__name__ + "("
        for t in self.transforms:
            format_string += "\n"
            format_string += "    {0}".format(t)
        format_string += "\n)"
        return format_string

# _pipeline = Compose(pipeline)

### get_sensor_data

In [10]:
from pathlib import Path

idx = 0
info = _nusc_infos[idx]
_root_path = Path(root_path)
res = {
            "lidar": {
                "type": "lidar",
                "points": None,
                "nsweeps": nsweeps,
                # "ground_plane": -gp[-1] if with_gp else None,
                "annotations": None,
            },
            "metadata": {
                "image_prefix": _root_path,
                "num_point_features": _num_point_features,
                "token": info["token"],
            },
            "calib": None,
            "cam": {},
            "mode": "val" if test_mode else "train",
            "virtual": virtual 
        }

# data, _ = _pipeline(res, info)
# return data

In [11]:
# for k, v in data.items():
#     print(f"key of data: {k}")

### compose 분석

In [12]:
pipeline

[{'type': 'LoadPointCloudFromFile', 'dataset': 'NuScenesDataset'},
 {'type': 'LoadPointCloudAnnotations', 'with_bbox': True},
 {'type': 'Preprocess',
  'cfg': {'mode': 'train',
   'shuffle_points': True,
   'global_rot_noise': [-0.3925, 0.3925],
   'global_scale_noise': [0.95, 1.05],
   'db_sampler': {'type': 'GT-AUG',
    'enable': False,
    'db_info_path': 'data/nuScenes/dbinfos_train_10sweeps_withvelo.pkl',
    'sample_groups': [{'car': 2},
     {'truck': 3},
     {'construction_vehicle': 7},
     {'bus': 4},
     {'trailer': 6},
     {'barrier': 2},
     {'motorcycle': 6},
     {'bicycle': 6},
     {'pedestrian': 2},
     {'traffic_cone': 2}],
    'db_prep_steps': [{'filter_by_min_num_points': {'car': 5,
       'truck': 5,
       'bus': 5,
       'trailer': 5,
       'construction_vehicle': 5,
       'traffic_cone': 5,
       'barrier': 5,
       'motorcycle': 5,
       'bicycle': 5,
       'pedestrian': 5}},
     {'filter_by_difficulty': [-1]}],
    'global_random_rotation_range_

In [13]:
# Compose는 __call__에서 아래 type의 클래스를 순서대로 초기화함 
for i in pipeline:
    print(i['type'])

LoadPointCloudFromFile
LoadPointCloudAnnotations
Preprocess
Voxelization
AssignLabel
Reformat


In [14]:
# __init__
_transforms = []
for transform in pipeline:
    transform = build_from_cfg(transform, PIPELINES)
    _transforms.append(transform)

2024-04-03 15:31:25,288 - INFO - {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}
2024-04-03 15:31:25,288 - INFO - [-1]
2024-04-03 15:31:27,046 - INFO - load 62964 traffic_cone database infos
2024-04-03 15:31:27,047 - INFO - load 65262 truck database infos
2024-04-03 15:31:27,047 - INFO - load 339949 car database infos
2024-04-03 15:31:27,047 - INFO - load 161928 pedestrian database infos
2024-04-03 15:31:27,047 - INFO - load 26297 ignore database infos
2024-04-03 15:31:27,048 - INFO - load 11050 construction_vehicle database infos
2024-04-03 15:31:27,048 - INFO - load 107507 barrier database infos
2024-04-03 15:31:27,048 - INFO - load 8846 motorcycle database infos
2024-04-03 15:31:27,048 - INFO - load 8185 bicycle database infos
2024-04-03 15:31:27,049 - INFO - load 12286 bus database infos
2024-04-03 15:31:27,049 - INFO - load 19202 trailer database infos
2024-04-03 15:31:27,907

In [15]:
# __call__(데이터를 불러 들일때 마다 실행)
# import time
# s = time.time()
# for t in _transforms:
#     s = time.time()
#     res, info = t(res, info)
#     e = time.time() - s
#     print(e)

### 순서대로 실행
1. LoadPointCloudFromFile
2. LoadPointCloudAnnotations
3. Preprocess
4. Voxelization
5. AssignLabel
6. Reformat

In [16]:
# 초기 res
res

{'lidar': {'type': 'lidar',
  'points': None,
  'nsweeps': 10,
  'annotations': None},
 'metadata': {'image_prefix': PosixPath('data/nuScenes'),
  'num_point_features': 5,
  'token': 'c3d61b8b66424ed9b1faa5114a0749e8'},
 'calib': None,
 'cam': {},
 'mode': 'train',
 'virtual': False}

In [17]:
# 초기 info (= _nusc_infos[idx])
info

{'lidar_path': '/mnt/hdd0/nuScenes/samples/LIDAR_TOP/n015-2018-08-03-15-00-36+0800__LIDAR_TOP__1533279699399021.pcd.bin',
 'cam_front_path': '/mnt/hdd0/nuScenes/samples/CAM_FRONT/n015-2018-08-03-15-00-36+0800__CAM_FRONT__1533279699362460.jpg',
 'cam_intrinsic': array([[1.26641720e+03, 0.00000000e+00, 8.16267020e+02],
        [0.00000000e+00, 1.26641720e+03, 4.91507066e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'token': 'c3d61b8b66424ed9b1faa5114a0749e8',
 'sweeps': [{'lidar_path': '/mnt/hdd0/nuScenes/sweeps/LIDAR_TOP/n015-2018-08-03-15-00-36+0800__LIDAR_TOP__1533279699349247.pcd.bin',
   'sample_data_token': 'fb4556116e844a559d9310d1073e9a1e',
   'transform_matrix': array([[ 9.99746766e-01,  2.24956666e-02,  5.89971196e-04,
            1.35732038e-02],
          [-2.24952384e-02,  9.99746689e-01, -7.22662026e-04,
           -1.73858174e-01],
          [-6.06078514e-04,  7.09207481e-04,  9.99999565e-01,
           -3.80722449e-03],
          [ 0.00000000e+00,  0.

In [18]:
# LoadPointCloudFromFile
loadPointCloudFromFileInstance = _transforms[0]
res, info = loadPointCloudFromFileInstance(res, info)

In [19]:
res

{'lidar': {'type': 'lidar',
  'points': array([[ -2.9725718 ,  -0.10636174,  -1.7744162 ,  17.        ],
         [ -3.1055346 ,  -0.10219607,  -1.7559084 ,  26.        ],
         [ -3.2495208 ,  -0.09824395,  -1.7383157 ,  33.        ],
         ...,
         [-39.66796   ,   3.3610184 ,   5.772566  ,  32.        ],
         [-36.988773  ,   3.0867887 ,   6.267912  ,   8.        ],
         [-37.118355  ,   3.1113088 ,   7.192914  ,  34.        ]],
        dtype=float32),
  'nsweeps': 10,
  'annotations': None,
  'times': array([[0.       ],
         [0.       ],
         [0.       ],
         ...,
         [0.2471931],
         [0.2471931],
         [0.2471931]], dtype=float32),
  'combined': array([[ -2.9725718 ,  -0.10636174,  -1.7744162 ,  17.        ,
            0.        ],
         [ -3.1055346 ,  -0.10219607,  -1.7559084 ,  26.        ,
            0.        ],
         [ -3.2495208 ,  -0.09824395,  -1.7383157 ,  33.        ,
            0.        ],
         ...,
         [

In [20]:
# LoadPointCloudAnnotations
LoadPointCloudAnnotationsInstance = _transforms[1]
res, info = LoadPointCloudAnnotationsInstance(res, info)

In [21]:
res

{'lidar': {'type': 'lidar',
  'points': array([[ -2.9725718 ,  -0.10636174,  -1.7744162 ,  17.        ],
         [ -3.1055346 ,  -0.10219607,  -1.7559084 ,  26.        ],
         [ -3.2495208 ,  -0.09824395,  -1.7383157 ,  33.        ],
         ...,
         [-39.66796   ,   3.3610184 ,   5.772566  ,  32.        ],
         [-36.988773  ,   3.0867887 ,   6.267912  ,   8.        ],
         [-37.118355  ,   3.1113088 ,   7.192914  ,  34.        ]],
        dtype=float32),
  'nsweeps': 10,
  'annotations': {'boxes': array([[-2.68844700e+01, -3.09273548e+01, -1.71092498e+00,
            2.22700000e+00,  3.74699998e+00,  1.80900002e+00,
            0.00000000e+00,  0.00000000e+00, -3.93140388e+00],
          [-2.11421261e+01,  7.67739296e+00, -9.41467822e-01,
            1.71800005e+00,  4.17600012e+00,  1.67700005e+00,
            0.00000000e+00,  0.00000000e+00,  9.80575383e-01],
          [-3.11682568e+01,  5.67910461e+01,  5.16336799e-01,
            1.82900000e+00,  4.29500008e+00,

In [22]:
# Preprocess
PreprocessInstance = _transforms[2]
res, info = PreprocessInstance(res, info)

In [23]:
PreprocessInstance.shuffle_points

True

In [24]:
res

{'lidar': {'type': 'lidar',
  'points': array([[-24.62008   ,   1.0767063 ,   2.298417  ,   9.        ,
            0.        ],
         [ -4.455129  ,  -1.2906953 ,   0.4294734 ,  31.        ,
            0.        ],
         [ -4.3853536 ,  -4.279363  ,  -1.8239943 ,  35.        ,
            0.20019817],
         ...,
         [ -0.34748253,   4.1688256 ,  -1.6967025 ,  11.        ,
            0.300277  ],
         [ -4.929438  ,  -4.7890797 ,  -1.8542825 ,  92.        ,
            0.10011506],
         [  2.7093234 ,  -0.9314638 ,  -1.802067  ,   4.        ,
            0.10011506]], dtype=float32),
  'nsweeps': 10,
  'annotations': {'gt_boxes': array([[ 1.52736206e+01,  3.29159050e+01,  3.41510713e-01,
            2.78781652e+00,  7.02920294e+00,  3.26412201e+00,
            3.27933240e+00,  5.69382429e+00, -2.63548231e+00],
          [ 1.55369701e+01,  2.07458096e+01,  2.53760338e-01,
            6.51838303e-01,  6.73533380e-01,  1.63008881e+00,
           -1.45304967e-02, -7

In [25]:
# Voxelization
VoxelizationInstance = _transforms[3]
res, info = VoxelizationInstance(res, info)

2024-03-28 11:03:57,432 - INFO - finding looplift candidates


In [26]:
res

{'lidar': {'type': 'lidar',
  'points': array([[-24.62008   ,   1.0767063 ,   2.298417  ,   9.        ,
            0.        ],
         [ -4.455129  ,  -1.2906953 ,   0.4294734 ,  31.        ,
            0.        ],
         [ -4.3853536 ,  -4.279363  ,  -1.8239943 ,  35.        ,
            0.20019817],
         ...,
         [ -0.34748253,   4.1688256 ,  -1.6967025 ,  11.        ,
            0.300277  ],
         [ -4.929438  ,  -4.7890797 ,  -1.8542825 ,  92.        ,
            0.10011506],
         [  2.7093234 ,  -0.9314638 ,  -1.802067  ,   4.        ,
            0.10011506]], dtype=float32),
  'nsweeps': 10,
  'annotations': {'gt_boxes': array([[ 1.52736206e+01,  3.29159050e+01,  3.41510713e-01,
            2.78781652e+00,  7.02920294e+00,  3.26412201e+00,
            3.27933240e+00,  5.69382429e+00, -2.63548231e+00],
          [ 1.55369701e+01,  2.07458096e+01,  2.53760338e-01,
            6.51838303e-01,  6.73533380e-01,  1.63008881e+00,
           -1.45304967e-02, -7

In [27]:
# AssignLabel
AssignLabelInstance = _transforms[4]
res, info = AssignLabelInstance(res, info)

In [28]:
res

{'lidar': {'type': 'lidar',
  'points': array([[-24.62008   ,   1.0767063 ,   2.298417  ,   9.        ,
            0.        ],
         [ -4.455129  ,  -1.2906953 ,   0.4294734 ,  31.        ,
            0.        ],
         [ -4.3853536 ,  -4.279363  ,  -1.8239943 ,  35.        ,
            0.20019817],
         ...,
         [ -0.34748253,   4.1688256 ,  -1.6967025 ,  11.        ,
            0.300277  ],
         [ -4.929438  ,  -4.7890797 ,  -1.8542825 ,  92.        ,
            0.10011506],
         [  2.7093234 ,  -0.9314638 ,  -1.802067  ,   4.        ,
            0.10011506]], dtype=float32),
  'nsweeps': 10,
  'annotations': {'gt_boxes': [array([[  6.1972537, -22.409037 ,  -1.4769034,   1.6981324,   3.9297662,
              1.5551422,  -3.201423 ,  -9.951798 ,   0.3064526]],
          dtype=float32),
    array([[ 1.5273621e+01,  3.2915905e+01,  3.4151071e-01,  2.7878165e+00,
             7.0292029e+00,  3.2641220e+00,  3.2793324e+00,  5.6938243e+00,
            -2.63548

In [29]:
# Reformat
ReformatInstance = _transforms[5]
res, info = ReformatInstance(res, info)

In [30]:
res

{'metadata': {'image_prefix': PosixPath('data/nuScenes'),
  'num_point_features': 5,
  'token': 'a402bff32036426bbde0c1c43042562f'},
 'points': array([[-24.62008   ,   1.0767063 ,   2.298417  ,   9.        ,
           0.        ],
        [ -4.455129  ,  -1.2906953 ,   0.4294734 ,  31.        ,
           0.        ],
        [ -4.3853536 ,  -4.279363  ,  -1.8239943 ,  35.        ,
           0.20019817],
        ...,
        [ -0.34748253,   4.1688256 ,  -1.6967025 ,  11.        ,
           0.300277  ],
        [ -4.929438  ,  -4.7890797 ,  -1.8542825 ,  92.        ,
           0.10011506],
        [  2.7093234 ,  -0.9314638 ,  -1.802067  ,   4.        ,
           0.10011506]], dtype=float32),
 'voxels': array([[[-2.4620081e+01,  1.0767063e+00,  2.2984171e+00,  9.0000000e+00,
           0.0000000e+00],
         [-2.4799477e+01,  1.0936470e+00, -2.2998712e+00,  8.0000000e+00,
           4.9779177e-02],
         [-2.4708918e+01,  1.0244765e+00, -2.3067186e+00,  8.0000000e+00,
       